In [6]:
from langchain_openai import ChatOpenAI
from tavily import TavilyClient
from dotenv import load_dotenv
import os

load_dotenv()

print("✅ All imports successful!")
print(f"✅ OpenAI Key loaded: {os.getenv('OPENAI_API_KEY')[:10]}...")
print(f"✅ Tavily Key loaded: {os.getenv('TAVILY_API_KEY')[:10]}...")


✅ All imports successful!
✅ OpenAI Key loaded: sk-proj-FH...
✅ Tavily Key loaded: tvly-dev-j...


In [7]:
# Test web search
from tavily import TavilyClient
import os

tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

# Test search
results = tavily.search("Indian IT sector latest news 2026", max_results=3)

for result in results['results']:
    print(f"\n📰 {result['title']}")
    print(f"🔗 {result['url']}")
    print(f"📝 {result['content'][:200]}...")



📰 Indian IT Sector Set to Outperform Markets in 2026, Says CIO
🔗 https://www.whalesbook.com/news/English/tech/Indian-IT-Sector-Set-to-Outperform-Markets-in-2026-Says-CIO/696f3fa4ad039ca929a6f379
📝 Indian IT stocks eye 2026 outperformance as AI matures and client spending rebounds. CIO Pankaj Murarka sees a gradual uptick....

📰 Indian IT Sector To Outperform Markets In 2026, Says Pankaj Murarka
🔗 https://www.ndtvprofit.com/markets/indian-it-sector-to-outperform-markets-in-2026-says-pankaj-murarka-10790704
📝 Pankaj Murarka expects the Indian IT sector to outperform markets in 2026 · AI maturation is driving improved client sentiment and growth...

📰 Budget 2026 signals India doubling down on its position as global ...
🔗 https://www.moneycontrol.com/news/business/information-technology/budget-2026-signals-india-doubling-down-on-its-position-as-global-tech-services-leader-it-industry-says-13803923.html
📝 Budget 2026 signals India doubling down on its position as global tech services lead

In [8]:
# Test OpenAI GPT
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

response = llm.invoke("Explain what a financial research agent does in 2 sentences.")

print("🤖 GPT Response:")
print(response.content)


🤖 GPT Response:
A financial research agent conducts in-depth analysis and gathers data on various financial markets, investment opportunities, and economic trends to provide insights and recommendations. They often utilize quantitative and qualitative methods to assess risks and returns, helping clients make informed financial decisions.


In [13]:
# Step 1: Research Plan Generator
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def generate_research_plan(query: str) -> dict:
    """Generate a research plan for the query"""
    
    prompt = ChatPromptTemplate.from_template("""
You are a financial research planning assistant for the IT sector.

User Query: {query}

Create a detailed research plan with 5-7 specific search queries that will comprehensively answer this query.
Each search should explore a different aspect.

Format your response as a numbered list of search queries only.

Example format:
1. [First search query]
2. [Second search query]
3. [Third search query]
...
""")
    
    response = llm.invoke(prompt.format_messages(query=query))
    plan = response.content
    
    print("📋 RESEARCH PLAN GENERATED:")
    print("=" * 60)
    print(plan)
    print("=" * 60)
    
    return {
        "query": query,
        "plan": plan,
        "searches_completed": 0,
        "findings": []
    }

# Test it
state = generate_research_plan("Analyze the current state of Indian IT services companies")


📋 RESEARCH PLAN GENERATED:
1. Current financial performance and revenue growth trends of major Indian IT services companies (e.g., TCS, Infosys, Wipro, HCL Technologies).
2. Impact of global economic conditions on the Indian IT services sector in 2023.
3. Analysis of emerging technologies and their adoption in Indian IT services (e.g., AI, cloud computing, cybersecurity).
4. Competitive landscape and market share analysis of Indian IT services firms compared to global players.
5. Recent mergers, acquisitions, and partnerships in the Indian IT services industry and their implications.
6. Talent acquisition and workforce trends in Indian IT services companies, including challenges and strategies.
7. Regulatory and policy changes affecting the Indian IT services sector in 2023.


In [14]:
# Step 2: Execute Research Searches
from tavily import TavilyClient
import os
import time

tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

def execute_research(state: dict, max_searches: int = 5) -> dict:
    """Execute web searches based on the research plan"""
    
    plan_lines = state['plan'].strip().split('\n')
    search_queries = [line.split('. ', 1)[1] if '. ' in line else line 
                     for line in plan_lines if line.strip() and line[0].isdigit()]
    
    # Limit to max_searches
    search_queries = search_queries[:max_searches]
    
    print(f"\n🔍 EXECUTING {len(search_queries)} RESEARCH SEARCHES...")
    print("=" * 60)
    
    all_findings = []
    
    for i, query in enumerate(search_queries, 1):
        print(f"\n[Search {i}/{len(search_queries)}] {query}")
        print("-" * 60)
        
        try:
            results = tavily.search(query, max_results=2)
            
            for result in results['results']:
                finding = {
                    'search_query': query,
                    'title': result['title'],
                    'url': result['url'],
                    'content': result['content']
                }
                all_findings.append(finding)
                
                print(f"✓ Found: {result['title']}")
                print(f"  URL: {result['url']}")
                print(f"  Preview: {result['content'][:150]}...")
            
            time.sleep(1)  # Be nice to the API
            
        except Exception as e:
            print(f"✗ Error: {e}")
    
    state['findings'] = all_findings
    state['searches_completed'] = len(search_queries)
    
    print("\n" + "=" * 60)
    print(f"✅ RESEARCH COMPLETE: {len(all_findings)} findings gathered")
    
    return state

# Execute the research
state = execute_research(state, max_searches=5)



🔍 EXECUTING 5 RESEARCH SEARCHES...

[Search 1/5] Current financial performance and revenue growth trends of major Indian IT services companies (e.g., TCS, Infosys, Wipro, HCL Technologies).
------------------------------------------------------------
✓ Found: Indian IT Firms Dominate Global IT Services 2026 | Saurav Kumar ...
  URL: https://www.linkedin.com/posts/saurav-kumar-b72835140_indianit-brandfinance-itservices-activity-7419828482567811072-mmVV
  Preview: The IT major posted revenue of ₹23,555 crore in Q3, marking a 5.5% year on year increase and a sequential rise of about 4%, even as...
✓ Found: Indian top IT firms set for another tepid quarter on weak US demand ...
  URL: https://finance.yahoo.com/news/indian-top-firms-set-another-051502915.html
  Preview: Infosys and HCLTech are forecast to report year-on-year revenue growth of ​about 8.1% and 4.6%, respectively, compared with 7.6% and 5.1% in the...

[Search 2/5] Impact of global economic conditions on the Indian IT service

In [15]:
# Step 3: Generate Research Report
from langchain_core.prompts import ChatPromptTemplate

def generate_report(state: dict) -> str:
    """Synthesize all findings into a comprehensive report"""
    
    # Compile all findings
    findings_text = ""
    for i, finding in enumerate(state['findings'], 1):
        findings_text += f"\n{i}. {finding['title']}\n"
        findings_text += f"   Source: {finding['url']}\n"
        findings_text += f"   {finding['content'][:300]}...\n"
    
    prompt = ChatPromptTemplate.from_template("""
You are a professional financial research analyst specializing in the IT sector.

Original Query: {query}

Research Plan Executed:
{plan}

Findings from {num_findings} sources:
{findings}

Create a comprehensive research report with the following sections:
1. Executive Summary (2-3 sentences)
2. Key Findings (bullet points)
3. Financial Performance Analysis
4. Market Trends & Outlook
5. Risks & Challenges
6. Conclusion

Be specific, cite key metrics, and provide actionable insights.
""")
    
    print("\n📝 GENERATING COMPREHENSIVE REPORT...")
    print("=" * 60)
    
    response = llm.invoke(prompt.format_messages(
        query=state['query'],
        plan=state['plan'],
        num_findings=len(state['findings']),
        findings=findings_text
    ))
    
    report = response.content
    
    print("\n" + "=" * 60)
    print("📊 RESEARCH REPORT")
    print("=" * 60)
    print(report)
    print("=" * 60)
    
    return report

# Generate the report
report = generate_report(state)



📝 GENERATING COMPREHENSIVE REPORT...

📊 RESEARCH REPORT
# Research Report: Current State of Indian IT Services Companies

## Executive Summary
The Indian IT services sector is experiencing moderate growth amid global economic uncertainties, particularly in the US and Europe. Major players like TCS, Infosys, and HCL Technologies are adapting to changing market dynamics, focusing on emerging technologies such as AI and cloud computing to drive future growth. However, challenges such as talent acquisition and regulatory changes pose risks to sustained performance.

## Key Findings
- **Revenue Growth**: TCS reported a 5.5% year-on-year revenue increase in Q3, while Infosys and HCLTech are projected to grow by 8.1% and 4.6%, respectively.
- **Global Economic Impact**: The Indian IT sector remains vulnerable to economic shocks in the US and Europe, leading to potential cost-cutting measures.
- **Emerging Technologies**: Significant investments are being made in AI, cloud computing, and cybe

In [16]:
# Advanced Feature 1: Save Reports to Files
from datetime import datetime
from pathlib import Path
import os

def save_report(report: str, query: str, format: str = "md") -> str:
    """Save research report to file"""
    
    # Create outputs directory if it doesn't exist
    output_dir = Path("outputs/reports")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Generate filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    safe_query = query[:50].replace(" ", "_").replace("/", "-")
    filename = f"{timestamp}_{safe_query}.{format}"
    filepath = output_dir / filename
    
    # Save report
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(f"# Research Report\n")
        f.write(f"**Query:** {query}\n")
        f.write(f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"**Sources:** {state['searches_completed']} searches, {len(state['findings'])} findings\n\n")
        f.write("---\n\n")
        f.write(report)
    
    print(f"\n💾 Report saved to: {filepath}")
    return str(filepath)

# Save the report we just generated
filepath = save_report(report, state['query'])



💾 Report saved to: outputs\reports\20260202_170517_Analyze_the_current_state_of_Indian_IT_services_co.md


In [17]:
# Advanced Feature 2: Intelligent Sector Router
from langchain_core.prompts import ChatPromptTemplate

def detect_sector(query: str) -> str:
    """Automatically detect which sector the query belongs to"""
    
    prompt = ChatPromptTemplate.from_template("""
You are a financial sector classification expert.

Analyze this query and determine which sector it belongs to:
Query: {query}

Available sectors:
- IT: Information Technology, software, tech services, SaaS, cloud computing
- PHARMA: Pharmaceutical, healthcare, biotech, drug development, medical devices
- GENERAL: If it doesn't clearly fit IT or PHARMA

Respond with ONLY one word: IT, PHARMA, or GENERAL
""")
    
    response = llm.invoke(prompt.format_messages(query=query))
    sector = response.content.strip().upper()
    
    print(f"🎯 SECTOR DETECTED: {sector}")
    return sector

# Test it
test_queries = [
    "Analyze Infosys financial performance",
    "Research Sun Pharma drug pipeline",
    "What are emerging trends in biosimilars?",
    "TCS vs Wipro comparison"
]

print("Testing Sector Detection:")
print("=" * 60)
for q in test_queries:
    print(f"\nQuery: {q}")
    sector = detect_sector(q)
    print("-" * 60)


Testing Sector Detection:

Query: Analyze Infosys financial performance
🎯 SECTOR DETECTED: IT
------------------------------------------------------------

Query: Research Sun Pharma drug pipeline
🎯 SECTOR DETECTED: PHARMA
------------------------------------------------------------

Query: What are emerging trends in biosimilars?
🎯 SECTOR DETECTED: PHARMA
------------------------------------------------------------

Query: TCS vs Wipro comparison
🎯 SECTOR DETECTED: IT
------------------------------------------------------------


In [18]:
# Advanced Feature 3: Deep Iterative Research with Adaptive Queries
def deep_research(state: dict, max_iterations: int = 10) -> dict:
    """
    Advanced research that adapts based on findings.
    Each iteration generates new questions based on previous results.
    """
    
    print(f"\n🔬 STARTING DEEP RESEARCH (up to {max_iterations} iterations)")
    print("=" * 60)
    
    all_findings = []
    iteration = 0
    
    # Initial searches from plan
    plan_lines = state['plan'].strip().split('\n')
    initial_queries = [line.split('. ', 1)[1] if '. ' in line else line 
                       for line in plan_lines if line.strip() and line[0].isdigit()]
    
    search_queue = initial_queries[:5]  # Start with first 5
    searched_queries = set()
    
    while search_queue and iteration < max_iterations:
        iteration += 1
        current_query = search_queue.pop(0)
        
        if current_query in searched_queries:
            continue
            
        searched_queries.add(current_query)
        
        print(f"\n[Iteration {iteration}] 🔍 {current_query}")
        print("-" * 60)
        
        try:
            results = tavily.search(current_query, max_results=2)
            
            iteration_findings = []
            for result in results['results']:
                finding = {
                    'iteration': iteration,
                    'search_query': current_query,
                    'title': result['title'],
                    'url': result['url'],
                    'content': result['content']
                }
                all_findings.append(finding)
                iteration_findings.append(finding)
                
                print(f"  ✓ {result['title'][:80]}...")
            
            # Generate follow-up questions based on findings
            if iteration < max_iterations - 2 and iteration_findings:
                followup = generate_followup_questions(current_query, iteration_findings)
                if followup:
                    search_queue.extend(followup[:2])  # Add top 2 follow-ups
                    print(f"  💡 Generated {len(followup)} follow-up queries")
            
            time.sleep(1)
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    state['findings'] = all_findings
    state['searches_completed'] = len(searched_queries)
    
    print("\n" + "=" * 60)
    print(f"✅ DEEP RESEARCH COMPLETE:")
    print(f"   • {iteration} iterations")
    print(f"   • {len(all_findings)} total findings")
    print(f"   • {len(searched_queries)} unique searches")
    print("=" * 60)
    
    return state

def generate_followup_questions(original_query: str, findings: list) -> list:
    """Generate intelligent follow-up questions based on findings"""
    
    findings_summary = "\n".join([f"- {f['title']}: {f['content'][:150]}" 
                                  for f in findings[:3]])
    
    prompt = ChatPromptTemplate.from_template("""
Based on this search query and results, generate 2 specific follow-up questions to deepen the research.

Original Query: {query}

Findings:
{findings}

Generate 2 specific, focused follow-up questions that dig deeper into interesting aspects mentioned in the findings.
Return only the questions, one per line, no numbering.
""")
    
    try:
        response = llm.invoke(prompt.format_messages(
            query=original_query,
            findings=findings_summary
        ))
        
        questions = [q.strip() for q in response.content.strip().split('\n') 
                    if q.strip() and not q.strip()[0].isdigit()]
        return questions[:2]
    except:
        return []

print("✅ Deep Research System Ready!")
print("This will perform 10+ iterative searches with adaptive follow-ups.")


✅ Deep Research System Ready!
This will perform 10+ iterative searches with adaptive follow-ups.


In [19]:
# Advanced Feature 4: User Approval Before Execution
def research_with_approval(query: str):
    """Complete research flow with user approval step"""
    
    print("\n" + "=" * 60)
    print("🎯 FINANCIAL RESEARCH AGENT")
    print("=" * 60)
    
    # Step 1: Detect sector
    sector = detect_sector(query)
    print(f"\n📊 Sector: {sector}")
    
    # Step 2: Generate plan
    state = generate_research_plan(query)
    
    # Step 3: Show plan and ask for approval
    print("\n⏸️  WAITING FOR APPROVAL")
    print("=" * 60)
    print("Review the research plan above.")
    print("\nOptions:")
    print("  1. Execute research (type 'yes' or 'y')")
    print("  2. Modify plan (type 'modify')")
    print("  3. Cancel (type 'no' or 'n')")
    
    # For notebook, we'll auto-proceed (in production, use input())
    approval = "yes"  # Change to input("Your choice: ") for interactive
    
    if approval.lower() in ['yes', 'y']:
        print("\n✅ APPROVED - Starting research...\n")
        
        # Step 4: Execute deep research
        state = deep_research(state, max_iterations=10)
        
        # Step 5: Generate report
        report = generate_report(state)
        
        # Step 6: Save report
        filepath = save_report(report, query)
        
        print(f"\n🎉 RESEARCH COMPLETE!")
        print(f"   Report saved: {filepath}")
        
        return state, report
    else:
        print("\n❌ Research cancelled")
        return None, None

print("✅ Approval Flow System Ready!")


✅ Approval Flow System Ready!


In [20]:
# TEST THE COMPLETE ADVANCED SYSTEM
test_query = "Analyze Indian IT sector performance and AI adoption trends 2026"

state, report = research_with_approval(test_query)



🎯 FINANCIAL RESEARCH AGENT
🎯 SECTOR DETECTED: IT

📊 Sector: IT
📋 RESEARCH PLAN GENERATED:
1. Current performance metrics of the Indian IT sector in 2026: revenue growth, market share, and key players.
2. Trends in AI adoption within the Indian IT sector: key technologies, use cases, and industry impact.
3. Government policies and initiatives supporting AI development in India: analysis of the National AI Strategy and funding programs.
4. Comparative analysis of AI adoption in the Indian IT sector versus global trends: benchmarks and best practices.
5. Challenges and barriers to AI implementation in the Indian IT sector: workforce skills, infrastructure, and regulatory issues.
6. Case studies of successful AI integration in Indian IT companies: lessons learned and future outlook.
7. Future projections for the Indian IT sector and AI adoption: expert opinions, market forecasts, and potential disruptions.

⏸️  WAITING FOR APPROVAL
Review the research plan above.

Options:
  1. Execute re

In [ ]:
y
